In [1]:
!pip install opencv-python-headless==4.5.4.60 fiftyone
!pip install fiftyone --no-binary fiftyone,voxel51-eta

     |████████████████████████████████| 47.6 MB 115 kB/s 
     |████████████████████████████████| 1.3 MB 36.8 MB/s 
     |████████████████████████████████| 564 kB 14.0 MB/s 
     |████████████████████████████████| 132 kB 70.7 MB/s 
     |████████████████████████████████| 79.9 MB 106 kB/s 
     |████████████████████████████████| 226 kB 62.0 MB/s 
     |████████████████████████████████| 47 kB 6.8 MB/s 
     |████████████████████████████████| 106 kB 9.5 MB/s 
     |████████████████████████████████| 55 kB 4.7 MB/s 
     |████████████████████████████████| 133 kB 58.0 MB/s 
     |████████████████████████████████| 29.2 MB 1.4 MB/s 
     |████████████████████████████████| 508 kB 75.4 MB/s 
     |████████████████████████████████| 13.2 MB 52.3 MB/s 
     |████████████████████████████████| 84 kB 3.8 MB/s 
     |████████████████████████████████| 68 kB 8.6 MB/s 
     |████████████████████████████████| 54 kB 3.7 MB/s 
     |████████████████████████████████| 80 kB 12.0 MB/s 
     |███████████████████

In [2]:
!pip install tensorflow torch torchvision umap-learn
!pip install ipywidgets>=7.5

     |████████████████████████████████| 88 kB 8.0 MB/s 
     |████████████████████████████████| 462 kB 61.9 MB/s 
     |████████████████████████████████| 1.1 MB 49.6 MB/s 
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82829 sha256=264b88b58df536aa5fa88b87058f207d1c7852bf936ee475b18c1bea38fa6285
  Stored in directory: /root/.cache/pip/wheels/b3/52/a5/1fd9e3e76a7ab34f134c07469cd6f16e27ef3a37aeff1fe821
  Created wheel for pynndescent: filename=pynndescent-0.5.7-py3-none-any.whl size=54286 sha256=47e20dc62a8e7ca278b21ed496b4301e430a9bdd95ab5bc19f55776cadf6753d
  Stored in directory: /root/.cache/pip/wheels/7f/2a/f8/7bd5dcec71bd5c669f6f574db3113513696b98f3f9b51f496c
Successfully built umap-learn pynndescent


In [ ]:
import os
import numpy as np
from PIL import Image
import io
import re
import zipfile

import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

import fiftyone as fo
import fiftyone.zoo as foz

# The Google Cloud Notebook product has specific requirements
IS_GOOGLE_CLOUD_NOTEBOOK = os.path.exists("/opt/deeplearning/metadata/env_version")

# Google Cloud Notebook requires dependencies to be installed with '--user'
USER_FLAG = ""
if IS_GOOGLE_CLOUD_NOTEBOOK:
    USER_FLAG = "--user"

PROJECT_ID = ""

if not os.getenv("IS_TESTING"):
    # Get your Google Cloud project ID from gcloud
    shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "bigbucket"  # @param {type:"string"}
from datetime import datetime
from google.colab import auth
auth.authenticate_user()
!echo "deb http://packages.cloud.google.com/apt gcsfuse-`lsb_release -c -s` main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!sudo apt-get -y -q update
!sudo apt-get -y -q install gcsfuse
!mkdir -p mush
bucket_name='medium_mush'
!gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 {bucket_name} mush

Project ID:  
deb http://packages.cloud.google.com/apt gcsfuse-bionic main
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2537  100  2537    0     0  27576      0 --:--:-- --:--:-- --:--:-- 27576
OK
Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 http://packages.cloud.google.com/apt gcsfuse-bionic InRelease [5,391 B]
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:5 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:6 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:7 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com

In [4]:
dataset = foz.load_zoo_dataset(
    "open-images-v6",
    split="train",
    max_samples=100,
    seed=51,
    shuffle=True,
)
session = fo.launch_app(dataset)

 100% |██████|    4.8Gb/4.8Gb [3.2s elapsed, 0s remaining, 1.6Gb/s]        


KeyboardInterrupt: ignored

In [5]:
dataset = foz.load_zoo_dataset(
    "open-images-v6",
    split=None,
    label_types='detections',
    classes = 'Mushroom',
    max_samples=2000,
    seed=51,
    shuffle=False,
    dataset_name="open-images-mush",
)

Only found 1782 (<2000) samples matching your requirements
 100% |█████████████████| 1782/1782 [1.4m elapsed, 0s remaining, 20.3 files/s]      
Only found 226 (<2000) samples matching your requirements
 100% |███████████████████| 226/226 [12.1s elapsed, 0s remaining, 19.6 files/s]      
Only found 64 (<2000) samples matching your requirements
 100% |█████████████████████| 64/64 [3.4s elapsed, 0s remaining, 18.7 files/s]      
Dataset info written to '/root/fiftyone/open-images-v6/info.json'
Loading 'open-images-v6' split 'train'
 100% |███████████████| 1782/1782 [10.1s elapsed, 0s remaining, 185.5 samples/s]      
Loading 'open-images-v6' split 'test'
 100% |█████████████████| 226/226 [1.1s elapsed, 0s remaining, 209.1 samples/s]         
Loading 'open-images-v6' split 'validation'
 100% |███████████████████| 64/64 [310.2ms elapsed, 0s remaining, 206.3 samples/s]      
Dataset 'open-images-mush' created


In [7]:
!cd /root/fiftyone

In [20]:
%cd Dataset

/root/fiftyone/OID/Dataset


In [25]:
!python convert_annotations.py

Currently in subdirectory: test
Converting annotations for class:  metadata
Traceback (most recent call last):
  File "convert_annotations.py", line 51, in <module>
    os.chdir("labels")
FileNotFoundError: [Errno 2] No such file or directory: 'labels'


In [13]:
!stat /root/fiftyone/open-images-v6/train/metadata/classes.csv

  File: /root/fiftyone/open-images-v6/train/metadata/classes.csv
  Size: 12011     	Blocks: 24         IO Block: 4096   regular file
Device: 25h/37d	Inode: 6690789     Links: 1
Access: (0644/-rw-r--r--)  Uid: (    0/    root)   Gid: (    0/    root)
Access: 2022-05-17 14:43:57.370183593 +0000
Modify: 2022-05-17 14:43:57.369183526 +0000
Change: 2022-05-17 14:43:57.369183526 +0000
 Birth: -


In [19]:
os.getcwd()

'/root/fiftyone/OID'

In [26]:
!git clone https://github.com/theAIGuysCode/OIDv4_ToolKit.git

Cloning into 'OIDv4_ToolKit'...
remote: Enumerating objects: 444, done.
remote: Total 444 (delta 0), reused 0 (delta 0), pack-reused 444
Receiving objects: 100% (444/444), 34.09 MiB | 25.46 MiB/s, done.
Resolving deltas: 100% (157/157), done.


In [28]:
%cd /root/OID

/root/OID


In [29]:
!pip3 install -r requirements.txt

     |████████████████████████████████| 3.9 MB 27.6 MB/s 
     |████████████████████████████████| 547 kB 46.0 MB/s 
  Attempting uninstall: rsa
    Found existing installation: rsa 4.8
    Uninstalling rsa-4.8:
      Successfully uninstalled rsa-4.8
  Attempting uninstall: docutils
    Found existing installation: docutils 0.17.1
    Uninstalling docutils-0.17.1:
      Successfully uninstalled docutils-0.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [8]:
fdic = {}
for folder in os.listdir('/content/mush/fungi/images/'):
    fdic[folder] = len(os.listdir('/content/mush/fungi/images/'+folder + '/'))
fdic

{'10000_Abortiporus_biennis': 74,
 '10025_Achroomyces_disciformis': 27,
 '10052_Agaricus_altipes': 20,
 '10056_Agaricus_arvensis': 228,
 '10057_Agaricus_augustus': 128,
 '10059_Agaricus_bernardii': 36,
 '10061_Agaricus_bisporus': 19,
 '10062_Agaricus_bitorquis': 100,
 '10065_Agaricus_campestris': 136,
 '10068_Agaricus_comtulus': 20,
 '10069_Agaricus_cupreobrunneus': 41,
 '10079_Agaricus_impudicus': 171,
 '10082_Agaricus_langei': 51,
 '10083_Agaricus_lanipes': 11,
 '10085_Agaricus_litoralis': 46,
 '10090_Agaricus_moelleri': 13,
 '10097_Agaricus_porphyrizon': 31,
 '10106_Agaricus_dulcidulus': 79,
 '10107_Agaricus_sylvaticus': 77,
 '10108_Agaricus_sylvicola': 84,
 '10111_Agaricus_subfloccosus': 20,
 '10112_Agaricus_subperonatus': 23,
 '10116_Agaricus_xanthodermus': 137,
 '10120_Agrocybe_arvalis': 23,
 '10123_Agrocybe_dura': 55,
 '10127_Agrocybe_pediades': 106,
 '10128_Agrocybe_praecox': 216,
 '10158_Aleuria_aurantia': 75,
 '10174_Aleurodiscus_amorphus': 13,
 '10225_Amanita_ceciliae': 73,


In [14]:
counts = pd.Series(fdic)

In [26]:
len(counts[counts['counts'].values > 50])

645

In [27]:
len(counts)

1394

In [25]:
# counts = counts.reset_index()
counts = counts.rename(columns={'index': 'species', 0:'counts'})

In [39]:
counts['genus'] = counts['species'].apply(lambda x: (x).split('_')[1])

In [48]:
freqdf = counts.groupby('genus', as_index=False).agg({'counts': 'sum', 'species': 'count'}).sort_values('counts')

In [56]:
freqdf['counts_per_species'] = round(freqdf['counts'] / freqdf['species'], 1)

In [66]:
freqdf.sort_values('species')

,genus,counts,species,counts_per_species
308,Physalacria,6,1,6.0
72,Clavariadelphus,63,1,63.0
23,Auriscalpium,63,1,63.0
36,Bulgaria,64,1,64.0
121,Elaphocordyceps,64,1,64.0
...,...,...,...,...
124,Entoloma,1564,38,41.2
206,Lactarius,2310,45,51.3
257,Mycena,3958,53,74.7
356,Russula,4346,78,55.7


In [65]:
freqdf.sort_values('counts_per_species').to_csv('species_freq.csv', index=False)

In [64]:
freqdf[freqdf['counts_per_species'] > 15]

,genus,counts,species,counts_per_species
267,Nodulisporium,16,1,16.0
332,Pterula,17,1,17.0
16,Ascodichaena,17,1,17.0
202,Lachnella,18,1,18.0
230,Loreleia,18,1,18.0
...,...,...,...,...
176,Hygrocybe,2161,28,77.2
206,Lactarius,2310,45,51.3
257,Mycena,3958,53,74.7
89,Cortinarius,4313,92,46.9
